# AIMS Data Ingestion Pipeline with DQ Gatekeeping

This notebook executes the data ingestion process with Data Quality checks.

## Purpose
- Load data files from local source directory
- Validate each file against its DQ config
- If PASSED: Process and mark as complete
- If FAILED: Quarantine and log for review
- Track processed files using watermarks

## Local Execution
This notebook runs entirely locally using the sample data in `data/Samples_LH_Bronze_Aims_26_parquet/`

In [ ]:
# Step 1: Install the library (if not already installed in the environment)
try:
    from notebookutils import mssparkutils
    IS_FABRIC = True
    print("Running in Microsoft Fabric")
except ImportError:
    IS_FABRIC = False
    print("Running Locally")

if not IS_FABRIC:
    %pip install ../dq_great_expectations/dq_package_dist/fabric_data_quality-1.1.2-py3-none-any.whl --force-reinstall
    # %pip install adlfs
else:
    # Fabric: Uncomment and adjust path if needed
    # %pip install /lakehouse/default/Files/libs/fabric_data_quality-1.1.2-py3-none-any.whl --force-reinstall
    pass

💻 Running Locally
Processing /home/sanmi/Documents/HS2/HS2_PROJECTS_2025/AIMS_LOCAL/dq_great_expectations/dq_package_dist/fabric_data_quality-1.1.2-py3-none-any.whl
Processing /home/sanmi/Documents/HS2/HS2_PROJECTS_2025/AIMS_LOCAL/dq_great_expectations/dq_package_dist/fabric_data_quality-1.1.2-py3-none-any.whl
  Using cached great_expectations-0.18.22-py3-none-any.whl.metadata (8.5 kB)
  Using cached great_expectations-0.18.22-py3-none-any.whl.metadata (8.5 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached pyyaml-6.0.3-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached pandas-2.3.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached pandas-2.3.3-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinu

In [ ]:
# Step 2: Configuration & Setup
import os
import json
from datetime import datetime
import glob
from pathlib import Path
from tqdm import tqdm

# Define Paths based on Environment
if IS_FABRIC:
    # Fabric Paths
    BASE_DIR = Path("/lakehouse/default/Files")
    DATA_PATH = BASE_DIR / "data/Samples_LH_Bronze_Aims_26_parquet"
    CONFIG_DIR = BASE_DIR / "dq_configs"
    STATE_DIR = BASE_DIR / "data/state"
    
    # Ensure state dir exists
    try:
        STATE_DIR.mkdir(exist_ok=True, parents=True)
    except OSError:
        print("Warning: Could not create state directory. Ensure you have write permissions.")

else:
    # Local Paths
    from dotenv import load_dotenv
    load_dotenv()
    
    BASE_DIR = Path(os.getenv("BASE_DIR", "/home/sanmi/Documents/HS2/HS2_PROJECTS_2025/AIMS_LOCAL"))
    DATA_PATH = BASE_DIR / os.getenv("DATA_PATH", "data/Samples_LH_Bronze_Aims_26_parquet")
    CONFIG_DIR = BASE_DIR / os.getenv("CONFIG_DIR", "dq_great_expectations/generated_configs")
    STATE_DIR = BASE_DIR / os.getenv("STATE_DIR", "data/state")
    STATE_DIR.mkdir(exist_ok=True, parents=True)

# Files for tracking state
WATERMARK_FILE = STATE_DIR / "watermarks.json"
DQ_LOG_FILE = STATE_DIR / "dq_logs.jsonl"

print(f"Configuration:")
print(f"   Environment: {'Fabric' if IS_FABRIC else 'Local'}")
print(f"   Data Source: {DATA_PATH}")
print(f"   DQ Configs: {CONFIG_DIR}")
print(f"   Watermarks: {WATERMARK_FILE}")
print(f"   DQ Logs: {DQ_LOG_FILE}")

✅ Configuration:
   Environment: Local
   Data Source: /home/sanmi/Documents/HS2/HS2_PROJECTS_2025/AIMS_LOCAL/data/Samples_LH_Bronze_Aims_26_parquet
   DQ Configs: /home/sanmi/Documents/HS2/HS2_PROJECTS_2025/AIMS_LOCAL/dq_great_expectations/generated_configs
   Watermarks: /home/sanmi/Documents/HS2/HS2_PROJECTS_2025/AIMS_LOCAL/data/state/watermarks.json
   DQ Logs: /home/sanmi/Documents/HS2/HS2_PROJECTS_2025/AIMS_LOCAL/data/state/dq_logs.jsonl


In [ ]:
# Step 3: Import DQ Libraries
from dq_framework import DataLoader, DataQualityValidator

print("Libraries imported successfully")

✅ Libraries imported successfully


In [ ]:
# Step 4: Watermark & Logging Helper Functions

def load_watermarks():
    """Load watermarks from JSON file."""
    if WATERMARK_FILE.exists():
        with open(WATERMARK_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_watermark(file_name):
    """Mark a file as processed."""
    watermarks = load_watermarks()
    watermarks[file_name] = datetime.now().isoformat()
    with open(WATERMARK_FILE, 'w') as f:
        json.dump(watermarks, f, indent=2)

def is_processed(file_name):
    """Check if file has already been processed."""
    watermarks = load_watermarks()
    return file_name in watermarks

def log_dq_result(file_name, status, score, details=None):
    """Append validation result to JSONL log file."""
    entry = {
        "timestamp": datetime.now().isoformat(),
        "file": file_name,
        "status": status,
        "score": score,
        "details": details or {}
    }
    with open(DQ_LOG_FILE, "a") as f:
        f.write(json.dumps(entry) + "\n")

def quarantine_file(file_path, reason):
    """Simulate moving file to quarantine."""
    print(f"   QUARANTINED: {os.path.basename(file_path)} -> Reason: {reason}")

print("Helper functions defined")

✅ Helper functions defined


In [ ]:
# Step 5: Execute Ingestion Pipeline with DQ Gatekeeping

# Discover all parquet files in source directory
source_files = list(DATA_PATH.glob("*.parquet"))
print(f"Starting Ingestion Pipeline for {len(source_files)} files...\n")

processed_count = 0
passed_count = 0
failed_count = 0
skipped_count = 0

# Use tqdm for progress bar
for file_path in tqdm(source_files, desc="Processing Files"):
    file_name = file_path.name
    
    # Check Watermark (Skip if already processed)
    if is_processed(file_name):
        print(f"Skipping {file_name} (Already Processed)")
        skipped_count += 1
        continue

    print(f"Processing: {file_name}...")
    
    try:
        # --- PHASE 1: DQ GATEKEEPING (Validation) ---
        # Look for corresponding validation config
        config_name = file_name.replace('.parquet', '_validation.yml')
        config_path = CONFIG_DIR / config_name
        
        if not config_path.exists():
            print(f"   Warning: No validation config found. Skipping DQ check.")
            validation_passed = True  # or False for strict mode
            score = 0.0
            failures = []
        else:
            # Load data and validate
            validator = DataQualityValidator(config_path=str(config_path))
            df_batch = DataLoader.load_data(str(file_path), sample_size=100000)
            result = validator.validate(df_batch)
            
            validation_passed = result['success']
            score = result['success_rate']
            failures = result.get('failed_expectations', [])
            
            # Log the result
            log_dq_result(
                file_name, 
                "PASSED" if validation_passed else "FAILED", 
                score, 
                {"failed_count": len(failures), "failures": failures[:5]}  # Log first 5 failures
            )

        # --- PHASE 2: ACTION ---
        if validation_passed:
            print(f"   DQ Passed (Score: {score:.1f}%). Ready for ingestion.")
            
            # In production, perform actual ingestion here:
            # - Write to Delta Lake / Silver layer
            # - Apply transformations
            # - Update metadata
            
            # Mark as processed
            save_watermark(file_name)
            print(f"   Marked as processed.")
            passed_count += 1
            
        else:
            print(f"   DQ FAILED (Score: {score:.1f}%). Blocked from ingestion.")
            quarantine_file(file_path, f"Failed {len(failures)} checks")
            failed_count += 1
            
        processed_count += 1
            
    except Exception as e:
        print(f"   Pipeline Error: {e}")
        log_dq_result(file_name, "ERROR", 0.0, {"error": str(e)})
        failed_count += 1

print(f"\n{'='*60}")
print(f"Pipeline Execution Complete")
print(f"{'='*60}")
print(f"Total Files: {len(source_files)}")
print(f"Processed: {processed_count}")
print(f"Passed DQ: {passed_count}")
print(f"Failed DQ: {failed_count}")
print(f"Skipped (Already Processed): {skipped_count}")
print(f"\nView results in Notebook 03 (Monitoring Dashboard)")

🚀 Starting Ingestion Pipeline for 68 files...



Processing Files:   0%|          | 0/68 [00:00<?, ?it/s]

⏭️  Skipping aims_attributegroups.parquet (Already Processed)
⏭️  Skipping aims_informationneedpropchngs.parquet (Already Processed)
📥 Processing: aims_organisations.parquet...


Calculating Metrics:   0%|          | 0/86 [00:00<?, ?it/s]

Processing Files:   4%|▍         | 3/68 [00:00<00:12,  5.02it/s]

...

❌ Validation FAILED: 1 checks failed
Processing Files:  96%|█████████▌| 65/68 [00:29<00:00,  3.96it/s]

   ❌ DQ FAILED (Score: 95.5%). Blocked from ingestion.
   🔒 QUARANTINED: aims_consentlinks.parquet -> Reason: Failed 1 checks
⏭️  Skipping aims_ua_optionvalues.parquet (Already Processed)
📥 Processing: aims_informationneedstatusupd.parquet...


Calculating Metrics:   0%|          | 0/73 [00:00<?, ?it/s]

❌ Validation FAILED: 1 checks failed
Processing Files: 100%|██████████| 68/68 [00:30<00:00,  2.23it/s]

   ❌ DQ FAILED (Score: 95.5%). Blocked from ingestion.
   🔒 QUARANTINED: aims_informationneedstatusupd.parquet -> Reason: Failed 1 checks
⏭️  Skipping aims_informationneeds.parquet (Already Processed)

🏁 Pipeline Execution Complete
Total Files: 68
Processed: 24
Passed DQ: 2
Failed DQ: 22
Skipped (Already Processed): 44

📊 View results in Notebook 03 (Monitoring Dashboard)
